In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import re

In [2]:
data=pd.read_csv("E:\Dataset\zomato/zomato.csv")
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [4]:
data.describe()

,votes
count,51717.000000
mean,283.697527
std,803.838853
min,0.000000
25%,7.000000
50%,41.000000
75%,198.000000
max,16832.000000



## Data Cleaning and Feature Engineering


In [5]:
#dropping some columns
data.drop(['url','dish_liked','phone'],axis=1, inplace=True)
data.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [6]:
#checking duplicate values
data.duplicated().sum()

43

In [7]:
#dropping dublicated values
data.duplicated().dropna(inplace=True)

In [8]:
#checking null values
data.isnull().sum()

address                           0
name                              0
online_order                      0
book_table                        0
rate                           7775
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     346
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

In [9]:
#dropping null values
data.dropna(how='any',inplace=True)

In [10]:
#Total no of null values
data.isnull().sum().sum()

0

In [11]:
#renaming some column names
data = data.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})

In [12]:
data.head(n=1)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari


In [13]:
#converting the dtype of the 2 columns to string dtype
# columns=["votes",'rate',"cost"]

# for column in columns:
#     data[column]=data[column].astype(str)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43533 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       43533 non-null  object
 1   name          43533 non-null  object
 2   online_order  43533 non-null  object
 3   book_table    43533 non-null  object
 4   rate          43533 non-null  object
 5   votes         43533 non-null  int64 
 6   location      43533 non-null  object
 7   rest_type     43533 non-null  object
 8   cuisines      43533 non-null  object
 9   cost          43533 non-null  object
 10  reviews_list  43533 non-null  object
 11  menu_item     43533 non-null  object
 12  type          43533 non-null  object
 13  city          43533 non-null  object
dtypes: int64(1), object(13)
memory usage: 5.0+ MB


In [15]:
#converting the cost into str to replace , with . and then changing its dtype to float
data['cost'] = data['cost'].astype(str)
data['cost'] = data['cost'].apply(lambda x: x.replace(',','.')) 
data['cost'] = data['cost'].astype(float) 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43533 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   address       43533 non-null  object 
 1   name          43533 non-null  object 
 2   online_order  43533 non-null  object 
 3   book_table    43533 non-null  object 
 4   rate          43533 non-null  object 
 5   votes         43533 non-null  int64  
 6   location      43533 non-null  object 
 7   rest_type     43533 non-null  object 
 8   cuisines      43533 non-null  object 
 9   cost          43533 non-null  float64
 10  reviews_list  43533 non-null  object 
 11  menu_item     43533 non-null  object 
 12  type          43533 non-null  object 
 13  city          43533 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 5.0+ MB


In [16]:
#removing /5 from the rate column
# data['rate'] = data['rate'].str.replace('/5', '', regex=False)
# data['rate']

In [17]:
#Removing '/5' from Rates
data = data.loc[data.rate !='NEW']
data = data.loc[data.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == str else x
data.rate = data.rate.apply(remove_slash).str.strip().astype('float')
data['rate'].head()

0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: float64

In [18]:
#replacing the values of online order and book table with true and false
data.name = data.name.apply(lambda x:x.title())
data.online_order.replace(('Yes','No'),(True, False),inplace=True)
data.book_table.replace(('Yes','No'),(True, False),inplace=True)


In [19]:
data.head(n=1)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari


In [20]:
# restaurants = list(data['name'].unique())
# data['Mean Rating'] = 0

# for i in range(len(restaurants)):
#     data['Mean Rating'][data['name'] == restaurants[i]] = data['rate'][data['name'] == restaurants[i]].mean()

In [21]:
restaurants = list(data['name'].unique())
data['Mean Rating'] = 0

for restaurant in restaurants:
    data.loc[data['name'] == restaurant, 'Mean Rating'] = data.loc[data['name'] == restaurant, 'rate'].mean()


In [22]:
scaler = MinMaxScaler(feature_range = (1,5))
data[['Mean Rating']] = scaler.fit_transform(data[['Mean Rating']]).round(2)
data.sample(3)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
5032,"20/2, Anjaneya Temple Street, YG Palya, Austin...",Little Chef,False,False,4.0,101,Richmond Road,Casual Dining,Chinese,550.0,"[('Rated 4.0', 'RATED\n Small and cozy place ...",[],Dine-out,Brigade Road,3.84
17085,"8, 1st Cross, Bharathi Layout, SG Palya, BTM, ...",Eat N Drink,False,False,3.8,87,BTM,Quick Bites,Fast Food,250.0,"[('Rated 3.0', ""RATED\n Menu is pretty basic....",[],Dine-out,Jayanagar,3.58
31327,"37, SC Road, KG Circle, Majestic, Bangalore",Blue Heaven Bar And Restaurant,False,False,3.2,6,Majestic,Bar,"Chinese, North Indian",750.0,"[('Rated 1.0', 'RATED\n this is a worst resta...",[],Drinks & nightlife,Lavelle Road,2.81


# Text Preprocessing

In [23]:
data[['reviews_list', 'cuisines']].sample(2)


,reviews_list,cuisines
32493,"[('Rated 1.0', 'RATED\n burned rice'), ('Rate...","North Indian, Kashmiri"
40282,"[('Rated 3.0', 'RATED\n Decent quality of foo...","South Indian, North Indian, Beverages"


In [24]:
## Lower Casing
data["reviews_list"] = data["reviews_list"].str.lower()
data[['reviews_list', 'cuisines']].sample(2)

,reviews_list,cuisines
8879,"[('rated 4.0', ""rated\n i was a little scepti...","American, Burger, Fast Food"
40195,"[('rated 1.0', 'rated\n we had ordered oreo s...","Beverages, Sandwich"


In [25]:
## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_punctuation(text))
data[['reviews_list', 'cuisines']].sample(2)

,reviews_list,cuisines
14480,rated 50 ratedn caramel hot chocolate at lãx...,Desserts
27973,rated 40 ratedn came across this new resto wh...,North Indian


In [26]:
#removal of stop words
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_stopwords(text))

In [27]:
## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

data["reviews_list"] = data["reviews_list"].apply(lambda text: remove_urls(text))

In [28]:
data[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
30004,rated 40 ratedn nice calm placenthey pretty go...,"Cafe, Desserts"
25215,rated 50 ratedn awesome food taste love itemsn...,"South Indian, Biryani, Chinese"
20179,rated 40 ratedn way better outlet thieves kamm...,"Bakery, Fast Food"
22667,rated 25 ratedn months back scouting onasadhya...,"Kerala, North Indian, Chinese"
22499,rated 50 ratedn mimansa classy place ambience ...,Cafe


In [29]:
# RESTAURANT NAMES:
restaurant_names = list(data['name'].unique())
restaurant_names

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point'

In [30]:

data = data[~data['name'].isin(['Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley', 'Bohra Bohra Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â©'])]

In [31]:
def get_top_words(column, top_nu_of_words, nu_of_word):
    
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    
    bag_of_words = vec.fit_transform(column)
    
    sum_words = bag_of_words.sum(axis=0)
    
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    return words_freq[:top_nu_of_words]

In [32]:
data.head(n=2)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,[],Buffet,Banashankari,3.97


In [33]:
data['votes'].describe()

count    41254.00000
mean       352.72393
std        884.26646
min          0.00000
25%         21.00000
50%         73.00000
75%        277.00000
max      16832.00000
Name: votes, dtype: float64

In [34]:
data.sample(2)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
14069,"195, Double Road, Indiranagar, Bangalore",Nagarjuna,False,False,4.2,1926,Indiranagar,Casual Dining,"Andhra, Biryani",850.0,rated 10 ratedn place good meals biryani spicy...,[],Delivery,Indiranagar,4.20
23019,"84, St.Antony Fairy Church",Seekers Cafe,False,False,3.6,25,BTM,Casual Dining,"Italian, Continental",500.0,rated 10 ratedn menu pictures highly deceiving...,[],Dine-out,Koramangala 4th Block,3.32


In [35]:
# x=data.groupby('name').count()['Mean Rating']>=4.7
# rest_with_high_rating_avg=x[x].index

# filtered_rating=data[data['name'].isin(rest_with_high_rating_avg)]
# filtered_rating.head(n=3)

# p_table=filtered_rating.pivot_table(index='name',columns='location' ,values='Mean Rating')
# p_table

In [36]:
data.shape

(41254, 15)

In [37]:
data.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city', 'Mean Rating'],
      dtype='object')

In [38]:
#Dropping some columns
data=data.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [39]:
# Randomly sample 60% of your dataframe
df_percent = data.sample(frac=0.5)

In [40]:
data.shape

(41254, 10)

In [41]:
#TF-IDF
df_percent.set_index('name', inplace=True)

In [42]:
indices = pd.Series(df_percent.index)

In [43]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])


In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
def recommend(name, cosine_similarities = cosine_similarities):   
    recommend_restaurant = []
    idx = indices[indices == name].index[0]
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)   
    top30_indexes = list(score_series.iloc[0:31].index)
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])  
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating'])
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating']][df_percent.index == each].sample()))
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(5)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    return df_new

In [ ]:
# HERE IS A RANDOM RESTAURANT. LET'S SEE THE DETAILS ABOUT THIS RESTAURANT:
df_percent[df_percent.index == 'La Casa'].head()

In [ ]:
recommend('Spice Elephant')

In [ ]:
# data['name'].unique

In [ ]:
#Names of the restaurents
restaurant_names_values = data['name'].tolist()
restaurant_names_values